## A/B тест

In [71]:
import pandas as pd
import numpy as np
from random import randint, shuffle
from itertools import groupby
import matplotlib.pyplot as plt
from scipy.stats import permutation_test

In [72]:
df = pd.read_csv('/content/exp_arppu_problem.tsv', sep = '\t')
df.loc[((df['value'] == 3790.0) & (df['action'] == 'confirmation'))]

,userid,timestamp,action,value,testids
853,user_10009,1661968551,confirmation,3790.0,13535;23346;23464;25661;34265
33381,user_10343,1659034056,confirmation,3790.0,13534;23345;23463;25662
337362,user_13454,1659890143,confirmation,3790.0,13535;23345;23463;25662


прочитаем файл

In [97]:
all_events = []
log_file = "/content/exp_arppu_problem.tsv"
with open(log_file, 'r') as f:
    f.readline()
    for line in f:
        parts = line.strip().split("\t")
        userid = parts[0]
        timestamp = int(parts[1])
        action = parts[2]
        value = float(parts[3])
        testids = parts[4]
        all_events.append((userid, timestamp, action, value, testids))

all_events.sort(key=lambda x: x[0])

Цель проведения AB-теста: посмотреть на влияние изменений в продукте на монетизацию

Посмотрим на поведение метрик ARPU и ARPPU. ARPU может быть "зашумлена" при наличии существенной доли пользователей, не приносящих доход. ARPPU же более чувствительна к реальным изменениям средней величины дохода.

Разделим на тестовую и контрольную выборки

In [98]:
exp_testid = 34265
control_testid = 34266
def with_testid(testid):
 return lambda e: str(testid) in e[4]
exp_events = list(filter(with_testid(exp_testid), all_events))
control_events = list(filter(with_testid(control_testid), all_events))

Нулевая гипотеза ($H_{0}$) - ARPU/ARPPU тестовой и контрольной выборок не отличаются

Альтернативная гипотеза ($H_{1}$) - ARPU/ARPPU тестовой и контрольной выборок отличаются

посчитаем arpu, p-value и покрасим тест

Зафиксируем $\alpha$ = 0.05

In [99]:
def color(p, test_v, control_v):
  if p > 0.05: return "grey"
  else:
    if test_v > control_v: return "green"
    else: return "red"

In [76]:
def revenues_by_user(events):
 return [
    sum(e[3] for e in user_events if e[2] == "confirmation")
    for user, user_events in groupby(events, lambda e: e[0])
 ]
exp_revenues = revenues_by_user(exp_events)
control_revenues = revenues_by_user(control_events)

In [77]:
np.random.seed(177)
def arpu(a):
 return np.mean(a)

def statistic(a, b):
 return arpu(a) - arpu(b)
result = permutation_test((exp_revenues, control_revenues), statistic, permutation_type="independent")
p_arpu = result.pvalue

In [78]:
np.random.seed(177)
def new(a, b):
  a_new = []
  for i in a:
    if i!=0:
      a_new.append(i)
  b_new = []
  for i in b:
    if i!=0:
      b_new.append(i)
  return a_new, b_new
def statistic(a, b):
  a_new, b_new = new(a, b)
  return arpu(a_new) - arpu(b_new)
result = permutation_test((exp_revenues, control_revenues), statistic, permutation_type="independent")
p_arppu = result.pvalue

In [80]:
a_new, b_new = new(exp_revenues, control_revenues)
col_arppu = color(p_arppu, arpu(a_new), arpu(b_new))
col_arpu = color(p_arpu, arpu(exp_revenues), arpu(control_revenues))

In [81]:
print(p_arpu, col_arpu)
print(p_arppu, col_arppu)

0.0014 red
0.042 green


Результаты:
```
0.0014 red
0.042 green
```

Для ARPU и для ARPPU на уровне значимости $\alpha$ = 0.05 $H_{0}$ отвергаем, так как p_value в обоих случаях < 0.05

Следовательно:

1.   есть статзначимое изменение в метриках в тестовой группе по сравнению с контрольной (метрики отличаются - подтверждение альтернативной гипотезы)
2.   Для ARPU вердикт red означает, что метрика «прокрасилась в минус» (в тестовой выборке есть статзначимое на уровне α изменение в отрицательную сторону), то есть ARPU меньше в тестовой
3.   Для ARPPU вердикт green означает, что метрика «прокрасилась в плюс» (в тестовой выборке есть статзначимое на уровне α изменение в положительную сторону), то есть ARPPU больше в тестовой

Можно сделать вывод, что в тестовой группе хоть и меньше средний доход с пользователя, но больше доход с платящего пользователя. То есть возможно меньше платящих пользователей, чем в контрольной, но каждый из них приносит нам больше денег, чем платящий пользователь из контрольной группы.


Выдвинем продуктовые гипотезы:

Такая ситуация возможна, если изменение в продукте

1.   привлекло больше оптовых клиентов (можно проверить, посчитав средний чек на заказ AOV, а не на пользователя)
2.   привлекло больше клиентов, покупающих дорогостоящие товары (нужны данные по количеству товаров в одном заказе)
3.   привело к снижению конверсии в оплату среди пользователей тестовой группы (посмотреть на ConvertionRate)




Проверим гипотезы 1 ($H_{0}$ - средние чеки в тестовой и контрольной выборках не отличаются)

2 ($H_{0}$ - средние стоимости добавленного в корзину товара в тестовой и контрольной выборках не отличаются)

и 3 ($H_{0}$ - средние чеки в тестовой и контрольной выборках не отличаются)

In [107]:
def revenues_by_order(events):
 return [
    e[3] for e in events if e[2] == "confirmation"
 ]
exp_revenues_order = revenues_by_order(exp_events)
control_revenues_order = revenues_by_order(control_events)

In [108]:
np.random.seed(177)
def aov(a):
 return np.mean(a)

def statistic(a, b):
 return aov(a) - aov(b)
result = permutation_test((exp_revenues_order, control_revenues_order), statistic, permutation_type="independent")
p_aov = result.pvalue
col_aov = color(p_aov, aov(exp_revenues_order), aov(control_revenues_order))

In [109]:
print(p_aov, col_aov)

0.0002 green


Отвергаем $H_{0}$ (средние чеки в тестовой и контрольной выборках не отличаются) с уровнем значимости $\alpha$ = 0.05

Получается, что aov(test) > aov(control), так как метрика прокрасилась в green.

In [111]:
def value_by_carts(events):
 return [
    e[3] for e in events if e[2] == "cart"
 ]
exp_carts = value_by_carts(exp_events)
control_carts = value_by_carts(control_events)

In [112]:
np.random.seed(177)
def mean(a):
 return np.mean(a)

def statistic(a, b):
 return mean(a) - mean(b)
result = permutation_test((exp_carts, control_carts), statistic, permutation_type="independent")
p_cart = result.pvalue
col_cart = color(p_cart, mean(exp_carts), mean(control_carts))

In [113]:
print(p_cart, col_cart)

0.0002 green


Отвергаем $H_{0}$ (средние стоимости добавленного в корзину товара в тестовой и контрольной выборках не отличаются) с уровнем значимости $\alpha$ = 0.05

Получается, что в тесте покупают более дорогие товары, чем в контрольной группе, так как метрика прокрасилась в green.

In [110]:
def checkouts_by_user(events):
  return [
    sum(1 for e in user_events if e[2] == "confirmation")
    for user, user_events in groupby(events, lambda e: e[0])
 ]
exp_conf = checkouts_by_user(exp_events)
control_conf = checkouts_by_user(control_events)
print(np.mean(exp_conf))
print(np.mean(control_conf))

np.random.seed(177)

def CR(a):
 return np.mean(a)

def statistic(a, b):
 return CR(b) - CR(a)
result = permutation_test((exp_conf, control_conf), statistic, permutation_type='independent', alternative='two-sided')
p_CR = result.pvalue
col_CR = color(p_CR, CR(exp_conf), CR(control_conf))
print(round(p_CR, 3), col_CR)

0.6126482213438735
1.0092213114754098
0.0 red


Отвергаем $H_{0}$ (средние чеки в тестовой и контрольной выборках не отличаются) с уровнем значимости $\alpha$ = 0.05

Получается, что в тестовой выборке конверсия упала, так как метрика прокрасилась в red



---
Выводы:


1.   Средний чек на заказ в тестовой группе больше, чем в контрольной.
2.   Средняя стоимость товара в корзине в тестовой группе больше, чем в контрольной.
3.   Конверсия в покупку в тестовой группе меньше, чем в контрольной.

Данные показывают, что возможно выдвинутые продуктовые гипотезы верны и изменение в продукте привело к изменению поведения пользователей


